# aoc 2024 d11

At first glance, they seem like normal stones: they're arranged in a perfectly straight line, and each stone has a number engraved on it.

The strange part is that every time you blink, the stones change.

Sometimes, the number engraved on a stone changes. Other times, a stone might split in two, causing all the other stones to shift over a bit to make room in their perfectly straight line.

As you observe them for a while, you find that the stones have a consistent behavior. Every time you blink, the stones each simultaneously change according to the first applicable rule in this list:

If the stone is engraved with the number 0, it is replaced by a stone engraved with the number 1.
If the stone is engraved with a number that has an even number of digits, it is replaced by two stones. The left half of the digits are engraved on the new left stone, and the right half of the digits are engraved on the new right stone. (The new numbers don't keep extra leading zeroes: 1000 would become stones 10 and 0.)
If none of the other rules apply, the stone is replaced by a new stone; the old stone's number multiplied by 2024 is engraved on the new stone.

In [ ]:
def split_even_digits(n:int):
    n_s = str(n)
    if len(n_s) %2 == 0:
        left,right = n_s[:len(n_s)//2],n_s[len(n_s)//2:]
        return [int(left),int(right)]
    else:
        return None

In [ ]:
split_even_digits(17)

[1, 7]

In [ ]:
split_even_digits(125)

In [ ]:
split_even_digits(2024)

[20, 24]

In [ ]:
def evolve_num(n:int) -> list[int]:
    if n == 0: return [1]
    elif lr := split_even_digits(n): return lr
    else: return [n*2024]

In [ ]:
evolve_num(125)

[253000]

In [ ]:
evolve_num(17)

[1, 7]

In [ ]:
def evolve_stonelist(stones:list[int]) -> list[int]:
    retval = []
    for n in stones:
        retval.extend( evolve_num(n))
    return retval

In [ ]:
e = [125,17]
e = evolve_stonelist(e)
e

[253000, 1, 7]

In [ ]:
e = evolve_stonelist(e)
e

[253, 0, 2024, 14168]

In [ ]:
e = evolve_stonelist(e)
e

[512072, 1, 20, 24, 28676032]

In [ ]:
def evolve_blinks(stones:list[int],blinks:int) -> list[int]:
    for _ in range(blinks):
        stones = evolve_stonelist(stones)
    return stones

In [ ]:
evolve_blinks([125,17],6)

[2097446912,
 14168,
 4048,
 2,
 0,
 2,
 4,
 40,
 48,
 2024,
 40,
 48,
 80,
 96,
 2,
 8,
 6,
 7,
 6,
 0,
 3,
 2]

In [ ]:
from aocd import get_data
inp = get_data(day=11, year=2024)

In [ ]:
inp

'70949 6183 4 3825336 613971 0 15 182'

In [ ]:
initial_stonelist = [int(n) for n in inp.split(' ')]
initial_stonelist

[70949, 6183, 4, 3825336, 613971, 0, 15, 182]

In [ ]:
len( evolve_blinks( [int(n) for n in inp.split(' ')], 25) )

185205

## Part 2

Compute it for 75 iterations.

Observations:

- stones never interact.
- so given a sequence S1 S3 S3 ... SN, then number of stones after q steps is sthe sum of the number of descendents individually.
- there are loops 0 -> 1 -> 2024 -> a:20 -> b:0
- if we can map all relevant loops, we can calculate the size growth over long trajectories analytically.
- can also use caching to save recomputing long trajectories.
- there's a recursive relationship defining long trajectories

Where S is a sequence of items S0 S1 S2 ... SN

```
len( evolve(blinks=n,S) = Sum_i[ len( evolve( blinks=n, Si )) ]
```

Could check for loops by seeing if there is a blink point after which all numbers have been seen before.

In [ ]:
def E(n,b) -> set[int]:
    return set( evolve_blinks([n],b) )
def L(n,b) -> int: return len( E(n,b) )

Recurrences appearing from 0

```
L(0,4) = 4
L(0,5) = 4
L(0,n) = L(0,n-4) + 2 * L(2,n-4) + L(4, n-4)
```

In [ ]:
def trail(start,max_blinks = 20):
    already_seen = set()
    already_seen.add(start)
    next_set = set([start])
    b = 0
    new_stones = set()
    while b <= max_blinks:
        fresh_stones = set(evolve_blinks(list(next_set),1))
        new_stones = fresh_stones - already_seen
        if len(new_stones) == 0:
            print(f"no new stones after {b} blinks")
            return b
        else:
            print(f"{b=}: {new_stones=}")
            b += 1
            next_set = new_stones
            already_seen = already_seen.union(new_stones)
    print(f"blinked {max_blinks} times but still finding new stones: {new_stones}")
    return b

In [ ]:
trail(0,30)

b=0: new_stones={1}
b=1: new_stones={2024}
b=2: new_stones={24, 20}
b=3: new_stones={2, 4}
b=4: new_stones={4048, 8096}
b=5: new_stones={40, 48, 80, 96}
b=6: new_stones={8, 9, 6}
b=7: new_stones={16192, 18216, 12144}
b=8: new_stones={32772608, 36869184, 24579456}
b=9: new_stones={9184, 3686, 3277, 2608, 9456, 2457}
b=10: new_stones={32, 36, 77, 84, 86, 56, 57, 26, 91, 94}
b=11: new_stones={3, 5, 7}
b=12: new_stones={6072, 14168, 10120}
b=13: new_stones={72, 20482880, 60, 28676032}
b=14: new_stones={2048, 2867, 2880, 6032}
b=15: new_stones={67, 28}
no new stones after 16 blinks


16

In [ ]:
trail(70949,30)

b=0: new_stones={143600776}
b=1: new_stones={290647970624}
b=2: new_stones={970624, 290647}
b=3: new_stones={624, 970, 290, 647}
b=4: new_stones={1262976, 1963280, 1309528, 586960}
b=5: new_stones={2556263424, 2650484672, 3973678720, 960, 586}
b=6: new_stones={63424, 1943040, 78720, 84672, 26504, 1186064, 39736, 25562}
b=7: new_stones={128370176, 3932712960, 53644096, 159329280, 171376128, 2400593536, 80425664, 51737488}
b=8: new_stones={259821236224, 4096, 322482462720, 346865283072, 12960, 24005, 93536, 5664, 7488, 8042, 5364, 5173, 39327}
b=9: new_stones={462720, 26231040, 189316864, 48586120, 79597848, 40, 42, 322482, 51, 53, 56, 236224, 283072, 64, 73, 74, 80, 88, 96, 259821, 346865}
b=10: new_stones={383177332736, 0, 2, 1, 4, 5, 3, 6, 7, 8, 9, 259, 1040, 7959, 283, 7848, 821, 2623, 322, 72, 462, 720, 346, 224, 865, 482, 6120, 236, 4858}
b=11: new_stones={1457280, 453376, 1661704, 10120, 10, 700304, 20, 23, 26, 8096, 18216, 48, 935088, 6072, 524216, 58, 59, 61, 332736, 16192, 3831

31

In [ ]:
trail(6183)

b=0: new_stones={83, 61}
b=1: new_stones={8, 1, 3, 6}
b=2: new_stones={16192, 2024, 12144, 6072}
b=3: new_stones={32772608, 24579456, 72, 20, 24, 60}
b=4: new_stones={0, 2, 4, 7, 3277, 2608, 9456, 2457}
b=5: new_stones={32, 8096, 77, 4048, 14168, 56, 57, 26, 94}
b=6: new_stones={96, 28676032, 5, 40, 9, 80, 48}
b=7: new_stones={10120, 18216, 2867, 6032}
b=8: new_stones={20482880, 67, 36869184, 28}
b=9: new_stones={2048, 9184, 2880, 3686}
b=10: new_stones={91, 36, 86, 84}
no new stones after 11 blinks


11

In [ ]:
[trail(n) for n in inp.split(' ')]

b=0: new_stones={'7094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970949709497094970

ValueError: Exceeds the limit (4300 digits) for integer string conversion: value has 5060 digits; use sys.set_int_max_str_digits() to increase the limit

So:

- we need to handle these numbers as strings
- some of the starting numebrs contain short cycles, other do not obviously.

**AI Prompt**

it seems to me this could be solve analytically by working out a recurrence felation for the number of new stones per blink. it would suffice to be able to calculate this per starting stone in a sequence. 

whats challenging to me is that one starting stone like 0 can trigger multiple loops at sifferent blink distances, so it will be intricate to find those loops and compute a relation whixh takes into account their differing lengths and distances from the starting stone.

**AI Response**

Yes, you've identified the key challenge here. Let me break down what we're seeing:

1. Each starting stone can indeed create multiple loops that start at different "depths" (number of blinks from start)
2. For example, with starting stone 0:
   - At blink 1: becomes 1
   - At blink 2: becomes 2024
   - At blink 3: splits into 20,24
   - Then these numbers start their own trajectories

Would you like to explore an approach to analyze these branching patterns systematically? We could:
1. Map out complete trajectories for small numbers first
2. Look for patterns in how the branches form
3. Try to find a mathematical way to combine the growth from different loops

Which of these would you like to investigate first?

**AI Prompt**

first I'd like to know what this kind if sustem is called.

**AI Response**

This type of system is called a "discrete dynamical system" - specifically, it's a branching or splitting dynamical system. It's similar to cellular automata, but with the key difference that each state can split into multiple new states.

The particular feature where a single value can split into multiple values makes it a "branching process" or "splitting process". These are commonly studied in population dynamics and probability theory.

Would you like me to explain more about any of these classifications, or shall we move on to analyzing the specific patterns in this system?

**AI Prompt**

tell me more about the main categories of such problems, where they usually appear in practice, and typical solution methods.

**AI Response**

Let me break this down into key categories and applications:

1. Branching Processes:
- Classic example: Population growth with reproduction
- Financial markets: Tree of possible price movements
- Nuclear chain reactions
- Typical methods: Probability generating functions, recursion formulas

2. Splitting Dynamical Systems:
- Chemical reactions where molecules split
- Data structure evolution (like B-trees)
- Typical methods: State space analysis, bifurcation theory

3. Discrete-Time Population Models:
- Species competition
- Virus spread
- Cell division
- Typical methods: Difference equations, matrix models

Would you like me to elaborate on any of these categories, or shall we discuss specific solution methods in more detail?

**AI Prompt**

let's discuss typical solution methods with analytical and otherwise

**AI Response**

I'll start with the main solution methods, beginning with analytical approaches:

1. Analytical Methods:
- Generating functions (for counting outcomes)
- Recurrence relations and difference equations
- State transition matrices
- Markov chain analysis (when probabilistic)

Would you like me to explain any of these in more detail before moving on to numerical and computational methods?

I find it's best to understand the analytical approaches first, since they often inform how we structure computational solutions.

```
0
1
2024
20,24
2,0,2,4

S(0,n) -> S(0,n-4) + 2*S(2,n-4) + S(4,n-4)

S(0,0)={0}
S(0,1)={1}
S(0,2)={2024}
S(0,3)={20,24}
S(0,4)={2,0,2,4}

```

Thought about this at length with paper. Here are my thoughts:

- The "line" of stones is a red herring. It's a bag of stones. the order of stones is irrelevant. They do not interact with each other. So all that matters is the COUNT of different KINDS of stones.
- If we have a function M that takes a bag of stones and gives a new bag of stones, then doubling the quantity of input stones will double the quantity of output stones.
- Also, if we two bgs of stones A and B, then applying our function to each bag individually and merging the results produces the same effect as applying our function to a merged bag A+B.
- iow, the function M is linear. So we can model a bag of stones as a vector v, where the v_i component tells the count of stones of type i.
- we can module time evolution as repeated application of M.

Consequent thoughts:
- this means we do not need to concretely represent the collection of stones. It suffices to represent the count of stones for every stone type. These counts can be represented in a single vector.
- the number might get very large, so it might not be possible to represent as ints in python, maybe?
- if we think of M as matrix, it will be a sparse matrix.
- rather than concentrating on time-evolving the initial bag of stones, we can focus on building up a representation of the time-evolution operation M. Then maybe we can optimize the work of using it to compute the composed time evolution operation.
- in particular, if we add entries to M every time a time evolve by one step, then we can keep track of if there are no new stones appearing, which would indicate that the future trajectory of stone creation/destruction may be understood in terms of loops.

Implications:
- we can define an abstraction which represents the linear operator M, which we build up gradually every time we need to apply it to a new item.

To formalize:
- let us represent a bag of stones with a vector, an instance of StoneBag
- a StoneBag has an int count for its components, which range from 0...inf
- the operator M is a linear operator. 
   - when M is applied to a unit component StoneBag, it returns a StoneBag with 0 or 1 values for every component.
   - we want a way to build up M by asserting new (src_unit:StoneBag, dst:StoneBag) pairs.
   - when we add new pairs, we also want to track the number of stones types which appear as destinations but not as srcs.

In [ ]:
from dataclasses import dataclass

Value = int
Count = int
StoneBag = dict[Value,int]

def unit_stonebag(value:Value) -> StoneBag:
    return {value:1}

In [ ]:
def add_stonebags(b1:StoneBag,b2:StoneBag) -> StoneBag:
    retval = {}
    for k in set(b1.keys()).union(set(b2.keys())):
        retval[k] = b1.get(k,0) + b2.get(k,0)
    return retval

In [ ]:
add_stonebags( unit_stonebag(1), unit_stonebag(2) )

{1: 1, 2: 1}

**AI Prompt**

Is the above implementation of vector addition correct?

**AI Response**

Yes, the implementation of `add_stonebags` is correct. It:
1. Creates an empty dictionary as the result
2. Iterates through all unique keys from both input bags
3. For each key, adds the counts from both bags (using get with default 0 for missing keys)
4. Returns the combined bag

This correctly implements vector addition for sparse vectors represented as dictionaries, where the keys are the indices and the values are the components.

Would you like to see some additional test cases to verify the implementation, or shall we move on to the next part of the system?

In [ ]:
import collections
import functools

def evolve(stonebag:StoneBag) -> StoneBag:
    countvecs = []
    for val,count in stonebag.items():
        minibag = collections.Counter(evolve_num(val))
        minibag_scaled = {k:v*count for k,v in minibag.items()}
        countvecs.append(minibag_scaled)
    result = functools.reduce(add_stonebags,countvecs)
    return result

In [ ]:
evolve( unit_stonebag( 0 ) )

{1: 1}

In [ ]:
evolve( add_stonebags( unit_stonebag( 0 ), unit_stonebag( 1 ) ) )

{2024: 1, 1: 1}

In [ ]:
evolve( evolve( unit_stonebag( 0 ) ) )

{2024: 1}

In [ ]:
evolve( evolve( evolve( unit_stonebag( 0 ) ) ) )

{20: 1, 24: 1}

In [ ]:
def evolve_bag_n(stonebag:StoneBag,iterations:int=1) -> StoneBag:
    result = stonebag
    for _ in range(iterations):
        result = evolve(result)
    return result

In [ ]:
evolve_bag_n( unit_stonebag( 0 ), 3 )

{20: 1, 24: 1}

In [ ]:
evolve_bag_n( unit_stonebag( 0 ), 4 )

{0: 1, 2: 2, 4: 1}

In [ ]:
from collections import Counter
c = Counter(evolve_blinks( initial_stonelist, blinks=5 ))
c

Counter({4048: 4,
         2: 3,
         0: 2,
         1: 2,
         8096: 2,
         1505856: 2,
         1683968: 2,
         586960: 1,
         1309528: 1,
         1963280: 1,
         1262976: 1,
         2457: 1,
         9456: 1,
         4: 1,
         3277: 1,
         2608: 1,
         6: 1,
         7: 1,
         32772608: 1,
         2024: 1,
         36869184: 1,
         24579456: 1,
         317173: 1,
         300224: 1,
         327988: 1,
         260864: 1,
         50826688: 1,
         10: 1,
         88: 1,
         80508648: 1,
         152399104: 1,
         20: 1,
         48: 1,
         28: 1,
         80: 1})

In [ ]:
len(c.keys()), sum(c.values())

(35, 45)

In [ ]:
len(  evolve_blinks( initial_stonelist, blinks=5 ) )

45

In [ ]:
initial_stones = [unit_stonebag(int(v)) for v in inp.strip().split(' ')]
initial_stones

[{70949: 1},
 {6183: 1},
 {4: 1},
 {3825336: 1},
 {613971: 1},
 {0: 1},
 {15: 1},
 {182: 1}]

In [ ]:
import functools
initial_bag = functools.reduce(lambda a,b: add_stonebags(a,b), initial_stones)
initial_bag

{0: 1, 613971: 1, 4: 1, 70949: 1, 182: 1, 6183: 1, 3825336: 1, 15: 1}

In [ ]:
r = evolve_bag_n(initial_bag, 5)
len(r), sum(r.values())

(35, 45)

In [ ]:
sum(evolve_bag_n(initial_bag, 2).values()), sum(Counter(evolve_blinks(initial_stonelist,2)).values())

(16, 16)

In [ ]:
evolve_bag_n(initial_bag, 2)

{96: 1,
 290647970624: 1,
 80064: 1,
 3: 1,
 1: 1,
 6: 1,
 2024: 1,
 10120: 1,
 1240712: 1,
 8: 1,
 80: 1,
 77424: 1,
 368: 2,
 1965304: 1}

In [ ]:
Counter(evolve_blinks(initial_stonelist,2))

Counter({2024: 2,
         368: 2,
         290647970624: 1,
         6: 1,
         1: 1,
         8: 1,
         3: 1,
         80: 1,
         96: 1,
         77424: 1,
         80064: 1,
         1240712: 1,
         1965304: 1,
         10120: 1})

In [ ]:
def count_stones(sb:StoneBag): return sum(sb.values())

In [ ]:
count_stones(evolve_bag_n(initial_bag,25))

185205

In [ ]:
count_stones(evolve_bag_n(initial_bag,75))

221280540398419